<a href="https://colab.research.google.com/github/stuartIane/MachineLearning/blob/main/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import subprocess

# Set environment variables
os.environ["CUDA"] = "cpu"  # Set CUDA to CPU (no GPU support)
os.environ["TORCH_INSTALL"] = "1"

# Install the torch package (assuming you are using pip)
subprocess.run(["pip", "install", "torch"])

# Download and install PyTorch (assuming you want the CPU version)
subprocess.run(["pip", "install", "torch-cpu"])

CompletedProcess(args=['pip', 'install', 'torch-cpu'], returncode=1)

In [1]:
pip install ISLP

In [4]:
from ISLP import load_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
default = load_data("Default")

In [6]:
default['default1'] = (default['default'] == "Yes").astype(int)

In [10]:
from sklearn.model_selection import train_test_split

x = default[['balance', 'income']].values # Include a column of ones for the intercept
x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)  # Add a column of ones for the intercept
y = default['default1'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 123)

In [ ]:
train_df_keys = {'balance': x_train[:,1], 'income': x_train[:,2], 'default': y_train }

train_df = pd.DataFrame(train_df_keys)

In [19]:
import torch
from torch.optim import LBFGS

In [28]:
# Assuming you have 'train' DataFrame with 'balance' and 'income' columns
x = torch.cat((torch.ones(train_df.shape[0], 1), torch.tensor(train_df[["balance", "income"]].values / 1000, dtype=torch.float64)), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["default"].values, dtype = torch.float64)
y.requires_grad = True

beta = torch.randn(3, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

Running on: cpu


In [29]:
# Define the loss function

def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

optimizer = LBFGS([beta], lr = 0.2)

def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    torch.nn.utils.clip_grad_value_(beta, 20)
    return value

In [31]:
for i in range(1, 10):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  print(f"{i}: {logl_value}")

1: nan
2: nan
3: nan
4: nan
5: nan
6: nan
7: nan
8: nan
9: nan


In [32]:
# Define the initial values
debugstart = torch.tensor([1, 2, 3], dtype=torch.float64, requires_grad=True)
x0 = torch.tensor([1, 1/2, 1/3], dtype=torch.float64, requires_grad=True)
y1 = torch.tensor([1], dtype=torch.float64, requires_grad=True)

# Calculate xbhat
xbhat = torch.mm(x0.unsqueeze(0), debugstart.unsqueeze(1))
xbhat.retain_grad()

# Calculate yhat
yhat = torch.sigmoid(xbhat)
yhat.retain_grad()

# Calculate the loss
loss = -torch.sum(y1 * torch.log(yhat) + (1 - y1) * torch.log(1 - yhat))
loss.backward()

# Print the tensors and their gradients
print("y:", y1)
print("x:", x0)
print("xbhat:", xbhat)
print("Gradients of xbhat (dloss/dp):", xbhat.grad)
print("yhat:", yhat)
print("Gradients of yhat (dloss/dyhat):", yhat.grad)
print("Loss:", loss.item())

y: tensor([1.], dtype=torch.float64, requires_grad=True)
x: tensor([1.0000, 0.5000, 0.3333], dtype=torch.float64, requires_grad=True)
xbhat: tensor([[3.]], dtype=torch.float64, grad_fn=<MmBackward0>)
Gradients of xbhat (dloss/dp): tensor([[-0.0474]], dtype=torch.float64)
yhat: tensor([[0.9526]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
Gradients of yhat (dloss/dyhat): tensor([[-1.0498]], dtype=torch.float64)
Loss: 0.04858735157374191
